# Google Firestore (Native Mode)

> [Firestore](https://cloud.google.com/firestore) is a serverless document-oriented database that scales to meet any demand. Extend your database application to build AI-powered experiences leveraging Firestore's LangChain integrations.

This notebook goes over how to use [Firestore](https://cloud.google.com/firestore) as a record manager for [langchain indexing](https://python.langchain.com/v0.1/docs/modules/data_connection/indexing/) your Vectorstore.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googleapis/langchain-google-firestore-python/blob/main/docs/record_manager.ipynb)

## Before You Begin

To run this notebook, you will need to do the following:

* [Create a Google Cloud Project](https://developers.google.com/workspace/guides/create-project)
* [Enable the Firestore API](https://console.cloud.google.com/flows/enableapi?apiid=firestore.googleapis.com)
* [Create a Firestore database](https://cloud.google.com/firestore/docs/manage-databases)

After confirmed access to database in the runtime environment of this notebook, filling the following values and run the cell before running example scripts.

In [ ]:
# @markdown Please specify a source for demo purpose.
COLLECTION_NAME = "test"  # @param {type:"CollectionReference"|"string"}

### 🦜🔗 Library Installation

The integration lives in its own `langchain-google-firestore` package, so we need to install it. For this notebook, we will also install `langchain-google-genai` to use Google Generative AI embeddings.

In [ ]:
%pip install --upgrade --quiet langchain langchain-google-firestore langchain-google-vertexai

**Colab only**: Uncomment the following cell to restart the kernel or use the button to restart the kernel. For Vertex AI Workbench you can restart the terminal using the button on top.

In [ ]:
# # Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

### ☁ Set Your Google Cloud Project
Set your Google Cloud project so that you can leverage Google Cloud resources within this notebook.

If you don't know your project ID, try the following:

* Run `gcloud config list`.
* Run `gcloud projects list`.
* See the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113).

In [ ]:
# @markdown Please fill in the value below with your Google Cloud project ID and then run the cell.

PROJECT_ID = "extensions-testing"  # @param {type:"string"}

# Set the project id
!gcloud config set project {PROJECT_ID}

### 🔐 Authentication

Authenticate to Google Cloud as the IAM user logged into this notebook in order to access your Google Cloud Project.

- If you are using Colab to run this notebook, use the cell below and continue.
- If you are using Vertex AI Workbench, check out the setup instructions [here](https://github.com/GoogleCloudPlatform/generative-ai/tree/main/setup-env).

In [ ]:
from google.colab import auth

auth.authenticate_user()

# Basic Usage

### Initialize FirestoreRecordManager

`FirestoreRecordManager` allows you to index your vectorstore in a Firestore database. You can use it to store references to embeddings from any model, including those from Google Generative AI.

In [ ]:
from langchain.indexes import index
from langchain_core.documents import Document
from langchain_google_firestore import FirestoreVectorStore, FirestoreRecordManager
from langchain_google_vertexai import VertexAIEmbeddings

namespace = f"firstore/{COLLECTION_NAME}"
record_manager = FirestoreRecordManager(namespace)

embedding = VertexAIEmbeddings(model_name="textembedding-gecko@latest")
vectorstore = FirestoreVectorStore(
    collection=COLLECTION_NAME,
    embedding_service=embedding
)

doc1 = Document(page_content="test-doc-1-content", metadata={"source": "test-doc-1.txt"})
doc2 = Document(page_content="test-doc-2-content", metadata={"source": "test-doc-2.txt"})
doc3 = Document(page_content="test-doc-3-content", metadata={"source": "test-doc-3.txt"})

results = index(
    [doc1, doc2, doc3],
    record_manager,
    vectorstore,
    cleanup="incremental",
    source_id_key="source",
)

print(results)